<a href="https://colab.research.google.com/github/abhijitsahoo0790/text_generation_using_LSTM/blob/master/main_text_generation_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Mount google drive to google Colab environment
from os.path import join
from google.colab import drive

ROOT = "/content/drive"
drive.mount(ROOT)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
"""
For creating a new project in GitHub, it will throw error if it is executed after the project dir is created
"""
ROOT = "/content/drive"
PROJ = "My Drive/Colab Notebooks/text_generation_using_LSTM" # This is a custom path.
PROJECT_PATH = join(ROOT, PROJ)
!mkdir "{PROJECT_PATH}"
!git clone https://github.com/abhijitsahoo0790/text_generation_using_LSTM.git "{PROJECT_PATH}"

In [4]:
ROOT = "/content/drive"
PROJ = "My Drive/Colab Notebooks/text_generation_using_LSTM" # This is a custom path.
PROJECT_PATH = join(ROOT, PROJ)
%cd "{PROJECT_PATH}"
!git pull origin master

/content/drive/My Drive/Colab Notebooks/text_generation_using_LSTM
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 27 (delta 14), reused 24 (delta 11), pack-reused 0
Unpacking objects: 100% (27/27), done.
From https://github.com/abhijitsahoo0790/text_generation_using_LSTM
 * branch            master     -> FETCH_HEAD
   6b9a15e..ff3aeba  master     -> origin/master
Updating 6b9a15e..ff3aeba
error: The following untracked working tree files would be overwritten by merge:
	data/republic.txt
Please move or remove them before you merge.
Aborting


In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import copy
import math
import os
from os.path import join
import sys
import traceback
import logging
logging.basicConfig(format='%(asctime)s %(levelname)-8s %(message)s', 
                    filename='log.txt', filemode='w', level=logging.DEBUG, 
                    datefmt='%Y-%m-%d %H:%M:%S')

Using TensorFlow backend.


In [ ]:
DATA_PATH = "/content/drive/My Drive/Colab Notebooks/text_generation_using_LSTM/data"
FILE_PATH = "wonderland.txt"
FULL_PATH_FILE = join(DATA_PATH, FILE_PATH)

# read the text file
f1 = open(FULL_PATH_FILE, 'r', encoding='utf-8')
raw_text = f1.read()
f1.close()

# convert text to lower case
raw_text = raw_text.lower()

# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
  seq_in = raw_text[i:i + seq_length]
  seq_out = raw_text[i + seq_length]
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])

n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))

# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

Total Characters:  163781
Total Vocab:  59
Total Patterns:  163681


In [ ]:
y.shape


(163681, 58)

In [ ]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])), )
model.add(Dropout(0.2))
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
163681/163681 [==============================] - 209s 1ms/step - loss: 2.9854

Epoch 00001: loss improved from inf to 2.98543, saving model to weights-improvement-01-2.9854.hdf5
Epoch 2/20
163681/163681 [==============================] - 203s 1ms/step - loss: 2.7968

Epoch 00002: loss improved from 2.98543 to 2.79681, saving model to weights-improvement-02-2.7968.hdf5
Epoch 3/20
163681/163681 [==============================] - 207s 1ms/step - loss: 2.7048

Epoch 00003: loss improved from 2.79681 to 2.70477, saving model to weights-improvement-03-2.7048.hdf5
Epoch 4/20
163681/163681 [==============================] - 202s 1ms/step - loss: 2.6348

Epoch 00004: loss improved from 2.70477 to 2.63485, saving model to weights-improvement-04-2.6348.hdf5
Epoch 5/20
163681/163681 [==============================] - 203s 1ms/step - loss: 2.5789

Epoch 00005: loss improved from 2.63485 to 2.57891, saving model to weights-improvement-05-2.5789.hdf5
Epoch 6/20
163681/163681 [=============

In [ ]:
# load the network weights
filename = "weights-improvement-20-2.0532.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [5]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
print("\n\n Generating Chars:")
# generate characters
for i in range(1000):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

NameError: ignored